In [10]:
%pip install langchain langchain_community
%pip install langchain_chroma
%pip install langchain_ollama
%pip install beautifulsoup4
%pip install aiohttp pyngrok
%pip install PyMuPDF openpyxl unstructured
%pip install lxml==4.9.2
%pip install --upgrade --quiet "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached pydantic_core-2.23.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.1
    Uninstalling pydantic_core-2.27.1:
      Successfully uninstalled pydantic_core-2.27.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.3
    Uninstalling pydantic-2.10.3:
      Successfully uninstalled pydant

In [3]:
!curl https://ollama.com/install.sh > ollama.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14703    0 14703    0     0   120k      0 --:--:-- --:--:-- --:--:--  120k


In [4]:
!sh ollama.sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################### 100.0%                                                         23.3%
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# Run in separate terminal
###       ollama serve

In [5]:
!ollama pull mannix/llama3.1-8b-lexi

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 739ad75e3eda...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 739ad75e3eda...   0% ▕                ▏  13 MB/4.7 GB                  pulling manifest 
pulling 739ad75e3eda...   1% ▕                ▏  47 MB/4.7 GB                  pulling manifest 
pulling 739ad75e3eda...   2% ▕                ▏ 112 MB/4.7 GB                  pulling manifest 
pulling 739ad75e3eda...   4% ▕                ▏ 193 MB/4.7 GB                  pulling manifest 
pulling 739ad75e3eda...   5% ▕                ▏ 233 MB/4.7 GB                  pulling manifest 
pulling 739ad75e3eda...   7% ▕█               ▏ 318 MB/4.7 GB                  pulling manifest 
pulling 739ad75e3eda...   9% ▕█      

In [6]:
!ls | grep pdf

Resume_ARS.pdf


In [7]:
%pip install --force-reinstall pydantic

  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached pydantic-2.10.3-py3-none-any.whl (456 kB)
Using cached pydantic_core-2.27.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: annotated-types
    Found existing installation: annotated-types 0.7.0
    Uninstalling annotated-types-0.7.0:
      Successfully uninstalled annotated-types-0.7.0
  Attemp

In [4]:
from pydantic import BaseModel, ValidationError
from typing import List
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import json
from datetime import datetime
from langchain_core.documents import Document
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Define the schema for topics
class TopicExtraction(BaseModel):
    """Model to extract topics from text."""
    topics: List[str]

class MCQSingle(BaseModel):
    """Model representing a single MCQ question."""
    topic: str
    page_numbers: str  # New field to store page numbers
    type: str = "MCQ"
    question: str
    options: List[str]
    correct_answer: str
    explanation: str

class MCQ(BaseModel):
    """Model containing a list of MCQSingle questions."""
    mcqs: List[MCQSingle]

class PDFProcessor:
    """Class to process a PDF, extract topics, generate MCQs, and save outputs."""

    def __init__(self, pdf_path, model_name):
        """Initialize the processor with PDF path and model name."""
        self.PDF_PATH = pdf_path
        self.MODEL_NAME = model_name
        self.llm = ChatOllama(model=self.MODEL_NAME, temperature=0.5)
        
        # Prompt templates
        self.topic_prompt_template = PromptTemplate(
            input_variables=["text"],
            template="""
The given context contains the contents of a page, do the following tasks:
1. Identify and extract the distinct, non-empty, and concise main topics in the content of the text.
2. Ensure the topics are well defined and relevant to the text and surrounded with double quotes in a single line.
Return the response as a JSON list of strings, e.g., ["Topic1", "Topic2", "Topic3"].

### CONTEXT:
{text}

TOPICS:
"""
        )
        self.mcq_prompt_template = PromptTemplate(
            input_variables=["topic", "schema", "context", "page_numbers"],
            template="""
Given the context:
{context}

Page Numbers: {page_numbers}

Generate 5 multiple-choice questions (MCQs) for the topic: "{topic}". 
Each question should have:
1. Three plausible but incorrect options and one correct option.
2. The number of options is always 4.
3. The options start with A), B), C), D).
4. No hint for the correct answer in the options.
5. An explanation key containing an explanation for the answer
Return the response as a list of JSON objects matching this schema:

{schema}

Return a json string containing a mcqs key with value as the list of the 5 MCQs, without any additional explanation or text.
"""
        )
        
        # Define OutputParsers
        self.topic_parser = PydanticOutputParser(pydantic_object=TopicExtraction)
        self.mcq_parser = PydanticOutputParser(pydantic_object=MCQSingle)
    
    def clean_json_response(self, response):
        """
        Cleans the LLM response by removing any text before the first '[' and after the last ']'.
        """
        start = response.find("[")
        end = response.rfind("]")
        if start != -1 and end != -1:
            return response[start:end+1]
        else:
            return ""
    
    def load_and_split_pdf(self):
        """Load the PDF, split it into documents, create embeddings, and store in ChromaDB."""
        loader = PyMuPDFLoader(self.PDF_PATH)
        self.documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter()
        split_docs = []
        for doc in self.documents:
            text = doc.page_content
            splits = text_splitter.split_text(text)
            for split in splits:
                # Create new Document with split text and original metadata
                new_doc = Document(page_content=split, metadata=doc.metadata)
                split_docs.append(new_doc)
        self.docs = split_docs
        # Create embeddings and add to ChromaDB in-memory
        embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        self.db = Chroma.from_documents(self.docs, embedding_function)
    
    def extract_topics_with_parser(self):
        """Extract topics from each split document and aggregate them."""
        all_topics = []
        for doc in self.docs:
            page_content = doc.page_content
            formatted_prompt = self.topic_prompt_template.format(text=page_content)
            response = self.llm.predict(formatted_prompt)
            print(f"###Context\n {page_content} \nEnd\n###Topics Response: {response} \n###End")
            try:
                topics = json.loads(response)
                if isinstance(topics, list):
                    all_topics.extend(topics)
                else:
                    print("Response is not a JSON list.")
            except json.JSONDecodeError as e:
                print(f"JSON parsing failed: {e}")
                start = response.find("[")
                end = response.rfind("]")
                if start != -1 and end != -1:
                    json_str = response[start:end+1]
                    try:
                        topics = json.loads(json_str)
                        all_topics.extend(topics)
                    except json.JSONDecodeError as e:
                        print(f"Failed to parse JSON list: {e}")
                else:
                    print("No JSON list found in response.")
        self.topics = list(set(all_topics))
    
    def generate_mcqs_with_parser(self, max_mcqs_per_topic=5):
        """Generate MCQs for each extracted topic using relevant document contexts."""
        all_mcqs = []
        for topic in self.topics:
            # Query ChromaDB for pages relevant to the topic
            similar_docs = self.db.similarity_search(topic, k=5)
            # Collect page numbers
            page_numbers = list(set([str(doc.metadata.get('page', '')) for doc in similar_docs]))
            page_numbers_str = ', '.join(page_numbers)
            # Combine the content of the retrieved pages
            combined_context = " ".join([doc.page_content for doc in similar_docs])
            # Generate MCQs using the combined context
            formatted_prompt = self.mcq_prompt_template.format(
                topic=topic,
                schema=self.mcq_parser.get_format_instructions(),
                context=combined_context,
                page_numbers=page_numbers_str  # Pass page numbers to the prompt
            )
            all_mcqs_topic = []
            response = self.llm.predict(formatted_prompt)
            if response.strip() == "":
                continue
            try:
                mcq_response = json.loads(response)
                if isinstance(mcq_response, dict) and "mcqs" in mcq_response:
                    mcqs_list = mcq_response["mcqs"]
                    if isinstance(mcqs_list, list):
                        for mcq_data in mcqs_list:
                            if not isinstance(mcq_data, dict):
                                continue
                            if 'correct_answer' in mcq_data and mcq_data['correct_answer'] not in mcq_data.get('options', []):
                                continue
                            try:
                                mcq = MCQSingle(**mcq_data)
                                mcq.topic = topic  # Ensure the topic is set
                                # Assign page_numbers to MCQSingle if available
                                mcq.page_numbers = page_numbers_str
                                all_mcqs.append(mcq)
                                all_mcqs_topic.append(mcq)
                                if len(all_mcqs_topic) >= max_mcqs_per_topic:
                                    break
                            except ValidationError as e:
                                print(f"Error validating MCQ data for topic '{topic}': {e}")
                    else:
                        print("Expected 'mcqs' to be a list.")
                else:
                    print("JSON response does not contain 'mcqs' key.")
            except json.JSONDecodeError as e:
                print(f"JSON parsing error for topic '{topic}': {e} : \n\n###Response:\n\n{response}")
        self.mcqs = all_mcqs
    
    def save_to_json(self, book_title):
        """Save extracted topics and generated MCQs to JSON files."""
        timestamp = datetime.utcnow().isoformat()
        topics_data = {
            "book_title": book_title,
            "total_topics": len(self.topics),
            "extraction_timestamp": timestamp,
            "main_topics": self.topics,
        }
        questions_data = {
            "metadata": {
                "generated_at": timestamp,
                "total_questions": len(self.mcqs),
                "book_title": book_title,
                "tool_used": "Ollama",
            },
            "questions": [
                {
                    "id": idx + 1,
                    "topic": mcq.topic,
                    "page_numbers": mcq.page_numbers,  # Include page_numbers in output
                    "type": mcq.type,
                    "question": mcq.question,
                    "options": mcq.options,
                    "explanation": mcq.explanation,
                    "correct_answer": mcq.correct_answer
                }
                for idx, mcq in enumerate(self.mcqs)
            ]
        }
        try:
            with open("topics.json", "w") as f:
                json.dump(topics_data, f, indent=4)
            print("Topics saved to topics.json")
        except Exception as e:
            print(f"Error saving topics: {e}")
        
        try:
            with open("questions.json", "w") as f:
                json.dump(questions_data, f, indent=4)
            print("Questions saved to questions.json")
        except Exception as e:
            print(f"Error saving questions: {e}")
    
    def process(self, book_title):
        """Execute the entire processing pipeline."""
        self.load_and_split_pdf()
        self.extract_topics_with_parser()
        # self.topics = ["Grades", "Education"]  # Uncomment for testing with specific topics
        print(f"###Extracted Topics: {self.topics}")
        self.generate_mcqs_with_parser()
        print(f"###Generated {len(self.mcqs)} MCQs.")
        self.save_to_json(book_title)
        print("Process completed!")

# Run the script
if __name__ == "__main__":
    processor = PDFProcessor(pdf_path="./Resume_ARS.pdf", model_name="mannix/llama3.1-8b-lexi")
    processor.process(book_title="Sample Book Title")

Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


###Context
 BISHWARAJ PAUL
# bishwaraj.paul98@gmail.com ï BishwarajPaul  +91 8134094503 § github.com/Verathagnus  Google Scholar
WORK EXPERIENCE
Machine Learning Engineer Intern
| Bahash-AI, Bahash Private Limited.
Dec 2023 – Present
• Deep Learning, Machine Learning and NLP.
Senior Software Development Engineer
| Brillio Technologies
June 2022 – June 2023
• MERN Stack and jBPM skillset
SKILLS
Coding: Python, JavaScript, LATEX
Databases: MySQL, SQLSERVER, MongoDB
Machine Learning: PYTHON (Tensorflow–PyTorch–NumPy–scikit learn)
Web Dev: HTML, CSS, JavaScript (TypeScript–NodeJS–ReactJS–Express–Redux Toolkit), Flask, AWS, Git, Docker, Nginx.
AWS Certified Cloud Practitioner: Issued by Amazon Web Services Training and Certification, September 19, 2022
KEY PROJECTS
WEB DEVELOPMENT
Website for Bahash AI–bahash.in
June, 2024
* Developed code for a website following the organizational requirements with the tech stack Flask, Python, HTML, CSS,
JavaScript, Bootstrap.
MACHINE LEARNING/NLP
Indic

Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


###Generated 12 MCQs.
Topics saved to topics.json
Questions saved to questions.json
Process completed!


In [5]:
!cat topics.json

{
    "book_title": "Sample Book Title",
    "total_topics": 4,
    "extraction_timestamp": "2024-12-03T19:38:57.293363",
    "main_topics": [
        "Skills",
        "Work Experience",
        "Education",
        "Key Projects"
    ]
}

In [6]:
!cat questions.json

{
    "metadata": {
        "generated_at": "2024-12-03T19:38:57.293363",
        "total_questions": 12,
        "book_title": "Sample Book Title",
        "tool_used": "Ollama"
    },
    "questions": [
        {
            "id": 1,
            "topic": "Skills",
            "page_numbers": "0",
            "type": "MCQ",
            "question": "What is the name of Bishwaraj Paul's first job?",
            "options": [
                "Google",
                "Amazon",
                "Bahash-AI"
            ],
            "explanation": "Bishwaraj Paul worked as a Machine Learning Engineer Intern at Bahash-AI.",
            "correct_answer": "Bahash-AI"
        },
        {
            "id": 2,
            "topic": "Skills",
            "page_numbers": "0",
            "type": "MCQ",
            "question": "What is the name of Bishwaraj Paul's second job?",
            "options": [
                "Microsoft",
                "Facebook",
                "Brillio Technologies"
   